In [1]:
import itertools
import numpy as np
import os

In [2]:
#np.set_printoptions(threshold=np.inf, linewidth=400)

In [3]:
# file_path = '/Users/SiSi/Documents/GitHub/BMI-203-HW3/BLOSUM50'
# BLOSUM50 = open(file_path, 'r')
# lines = BLOSUM50.readlines()
# BLOSUM50.close()
# #print(lines[0:4])
# li = 0
# while lines[li][0] == '#':
#     li = li + 1
# headerline = lines[li]
# score_lines = lines[li+1:]

# #score_string = ''.join(score_lines)
# #print(score_string)
# #gives error - score_matrix = np.array([[int(j) for j in i.split(' ')] for i in score_string.splitlines()])
# #score_matrix = np.fromstring(score_matrix, dtype=int, sep=" ")
# score_matrix = np.zeros(())
# matsize = len(score_lines)
# for srow in range(matsize):
#     score_matrix[srow,:] = np.fromstring(score_lines[srow], dtype=int, sep=" ")
    
# print(score_matrix)    
    
    
#     #numpy.fromstring(string, dtype=float, count=-1, sep='')

In [4]:
file_path = '/Users/SiSi/Documents/GitHub/BMI-203-HW3/BLOSUM50'
BLOSUM50 = np.loadtxt(file_path, dtype=int, comments='#', skiprows=7)#, delimiter=' \n', ndmin = 2)
print(BLOSUM50)
#BLOSUM50[BLOSUM50 < 0] = 0 #replacing all negative values with 0 for Smith-Waterman
#zcol = np.zeros((BLOSUM50.shape[0],1))
#print(BLOSUM50.shape)
#print(zrow.shape)
#BLOSUM50z = np.concatenate((zcol, BLOSUM50), axis = 1)
#zrow = np.zeros((1,BLOSUM50z.shape[1]))
#print(BLOSUM50z.shape)
#print(zrow.shape)
#BLOSUM50zz = np.concatenate((zrow, BLOSUM50z), axis = 0)
#print(BLOSUM50zz)
#print(BLOSUM50zz.shape)







#numpy.loadtxt(fname, dtype=<class 'float'>, comments='#', delimiter=None, converters=None, skiprows=0, usecols=None, unpack=False, ndmin=0, encoding='bytes')[source]

[[ 5 -2 -1 -2 -1 -1 -1  0 -2 -1 -2 -1 -1 -3 -1  1  0 -3 -2  0 -2 -1 -1 -5]
 [-2  7 -1 -2 -4  1  0 -3  0 -4 -3  3 -2 -3 -3 -1 -1 -3 -1 -3 -1  0 -1 -5]
 [-1 -1  7  2 -2  0  0  0  1 -3 -4  0 -2 -4 -2  1  0 -4 -2 -3  4  0 -1 -5]
 [-2 -2  2  8 -4  0  2 -1 -1 -4 -4 -1 -4 -5 -1  0 -1 -5 -3 -4  5  1 -1 -5]
 [-1 -4 -2 -4 13 -3 -3 -3 -3 -2 -2 -3 -2 -2 -4 -1 -1 -5 -3 -1 -3 -3 -2 -5]
 [-1  1  0  0 -3  7  2 -2  1 -3 -2  2  0 -4 -1  0 -1 -1 -1 -3  0  4 -1 -5]
 [-1  0  0  2 -3  2  6 -3  0 -4 -3  1 -2 -3 -1 -1 -1 -3 -2 -3  1  5 -1 -5]
 [ 0 -3  0 -1 -3 -2 -3  8 -2 -4 -4 -2 -3 -4 -2  0 -2 -3 -3 -4 -1 -2 -2 -5]
 [-2  0  1 -1 -3  1  0 -2 10 -4 -3  0 -1 -1 -2 -1 -2 -3  2 -4  0  0 -1 -5]
 [-1 -4 -3 -4 -2 -3 -4 -4 -4  5  2 -3  2  0 -3 -3 -1 -3 -1  4 -4 -3 -1 -5]
 [-2 -3 -4 -4 -2 -2 -3 -4 -3  2  5 -3  3  1 -4 -3 -1 -2 -1  1 -4 -3 -1 -5]
 [-1  3  0 -1 -3  2  1 -2  0 -3 -3  6 -2 -4 -1  0 -1 -3 -2 -3  0  1 -1 -5]
 [-1 -2 -2 -4 -2  0 -2 -3 -1  2  3 -2  7  0 -3 -2 -1 -1  0  1 -3 -1 -1 -5]
 [-3 -3 -4 -5 -2 -4 -3 -4

In [5]:
sequence_path = '/Users/SiSi/Documents/GitHub/BMI-203-HW3/sequences/prot-0004.fa'
with open (sequence_path, "r") as file:
    seq_1=file.read()
    seq_1 = seq_1[seq_1.find("\n")+1:].replace('\n', '')
print(seq_1)

SLEAAQKSNVTSSWAKASAAWGTAGPEFFMALFDAHDDVFAKFSGLFSGAAKGTVKNTPEMAAQAQSFKGLVSNWVDNLDNAGALEGQCKTFAANHKARGISAGQLEAAFKVLSGFMKSYGGDEGAWTAVAGALMGEIEPDM


In [6]:
sequence_path = '/Users/SiSi/Documents/GitHub/BMI-203-HW3/sequences/prot-0008.fa'
with open (sequence_path, "r") as file:
    seq_2=file.read()
    seq_2 = seq_2[seq_2.find("\n")+1:].replace('\n', '')
print(seq_2)

ANKTRELCMKSLEHAKVDTSNEARQDGIDLYKHMFENYPPLRKYFKSREEYTAEDVQNDPFFAKQGQKILLACHVLCATYDDRETFNAYTRELLDRHARDHVHMPPEVWTDFWKLFEEYLGKKTTLDEPTKQAWHEIGREFAKEINK


In [7]:
print(len(seq_1))
print(len(seq_2))
print(BLOSUM50.shape)

142
147
(24, 24)


In [8]:
AA_list = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", 
           "M", "F", "P", "S", "T", "W", "Y", "V", "B", "Z", "X", "x"]

similarity_matrix_index = {}
for i, aa in enumerate(AA_list):
    similarity_matrix_index[aa] = i


In [28]:
def load_sim_matrix(name):
    pathway = os.path.join('.', name) #on Github, needs to be changed to '..'
    if name == 'BLOSUM50' or name == 'BLOSUM62':
        sim_matrix = np.loadtxt(os.path.abspath(pathway), dtype=int, comments='#', skiprows=7)
    elif name == 'MATIO':
        sim_matrix = np.loadtxt(os.path.abspath(pathway), dtype=int, comments='#', skiprows=3)
    else:
        sim_matrix = np.loadtxt(os.path.abspath(pathway), dtype=int, comments='#', skiprows=10)
        
    AA_list = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", 
           "M", "F", "P", "S", "T", "W", "Y", "V", "B", "Z", "X", "x"]
    similarity_matrix_index = {}
    for i, aa in enumerate(AA_list):
        similarity_matrix_index[aa] = i
    
    return sim_matrix, similarity_matrix_index
    

In [10]:
print(load_sim_matrix("BLOSUM50"))

(array([[ 5, -2, -1, -2, -1, -1, -1,  0, -2, -1, -2, -1, -1, -3, -1,  1,
         0, -3, -2,  0, -2, -1, -1, -5],
       [-2,  7, -1, -2, -4,  1,  0, -3,  0, -4, -3,  3, -2, -3, -3, -1,
        -1, -3, -1, -3, -1,  0, -1, -5],
       [-1, -1,  7,  2, -2,  0,  0,  0,  1, -3, -4,  0, -2, -4, -2,  1,
         0, -4, -2, -3,  4,  0, -1, -5],
       [-2, -2,  2,  8, -4,  0,  2, -1, -1, -4, -4, -1, -4, -5, -1,  0,
        -1, -5, -3, -4,  5,  1, -1, -5],
       [-1, -4, -2, -4, 13, -3, -3, -3, -3, -2, -2, -3, -2, -2, -4, -1,
        -1, -5, -3, -1, -3, -3, -2, -5],
       [-1,  1,  0,  0, -3,  7,  2, -2,  1, -3, -2,  2,  0, -4, -1,  0,
        -1, -1, -1, -3,  0,  4, -1, -5],
       [-1,  0,  0,  2, -3,  2,  6, -3,  0, -4, -3,  1, -2, -3, -1, -1,
        -1, -3, -2, -3,  1,  5, -1, -5],
       [ 0, -3,  0, -1, -3, -2, -3,  8, -2, -4, -4, -2, -3, -4, -2,  0,
        -2, -3, -3, -4, -1, -2, -2, -5],
       [-2,  0,  1, -1, -3,  1,  0, -2, 10, -4, -3,  0, -1, -1, -2, -1,
        -2, -3,  2, -4,

In [11]:
print(load_sim_matrix("MATIO"))

(array([[ 0,  1,  1, -6, -2,  8, -7,  4, -6, -1, -1,  4,  1,  2, -1, -8,
         7,  7,  4, -8, -1,  8, -1,  9],
       [ 1, -1,  1, -4, -5,  3,  2,  3,  1,  0, -7, -2, -9,  8, -7,  2,
        -4, -4,  2,  0,  0,  6, -7,  8],
       [ 1,  1, -6,  1,  5,  2,  4, -3, -5, -8,  8,  9,  4,  7,  8,  1,
        -2, -7,  4,  1,  1,  0,  7,  0],
       [-6, -4,  1,  6,  0,  0,  0, -8, -9,  5, -9,  3,  6,  7,  9, -9,
         4,  0,  6,  5, -2,  3,  4,  0],
       [-2, -5,  5,  0, -2,  2,  1,  2, -2,  2,  0, -6,  3, -1, -7,  3,
        -4,  7, -3, -1,  8,  1,  3,  9],
       [ 8,  3,  2,  0,  2, -2, -4,  4,  5,  3, -3,  4,  6, -4, -9,  1,
         5,  1, -7,  4,  2,  4,  0, -7],
       [-7,  2,  4,  0,  1, -4, -6,  5,  9,  0, -1, -4,  3,  5,  3,  6,
        -6,  6,  6, -1, -8,  5,  3, -9],
       [ 4,  3, -3, -8,  2,  4,  5,  0,  2, -7, -8,  2, -4,  9,  3, -4,
         5, -9,  5, -2,  8, -5,  9, -7],
       [-6,  1, -5, -9, -2,  5,  9,  2,  7,  5, -4, -8, -4,  3,  8,  0,
         1, -7, -1,  0,

In [12]:
PAM250, PAM250_index = load_sim_matrix("PAM250")

In [13]:
print(similarity_matrix_index)

{'A': 0, 'R': 1, 'N': 2, 'D': 3, 'C': 4, 'Q': 5, 'E': 6, 'G': 7, 'H': 8, 'I': 9, 'L': 10, 'K': 11, 'M': 12, 'F': 13, 'P': 14, 'S': 15, 'T': 16, 'W': 17, 'Y': 18, 'V': 19, 'B': 20, 'Z': 21, 'X': 22, '*': 23}


In [59]:
def matrix(a, b, match_score_similarity_matrix, similarity_matrix_index, gap_cost): #could not figure out implementation of elimination cost
    H = np.zeros((len(a) + 1, len(b) + 1), np.int)

    for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
        match = H[i - 1, j - 1] + match_score_similarity_matrix[similarity_matrix_index[a[i-1]], similarity_matrix_index[b[j-1]]]
        delete = H[i - 1, j] - gap_cost
        insert = H[i, j - 1] - gap_cost
        H[i, j] = max(match, delete, insert, 0)
    return H

In [64]:
matrix('ARN','ARN',BLOSUM50, BLOSUM50_i, 2)

array([[ 0,  0,  0,  0],
       [ 0,  5,  3,  1],
       [ 0,  3, 12, 10],
       [ 0,  1, 10, 19]])

In [17]:
score_matrix = matrix(seq_1, seq_2, BLOSUM50, similarity_matrix_index)
print(score_matrix)

[[  0   0   0 ...   0   0   0]
 [  0   1   1 ...   0   1   0]
 [  0   0   0 ...   2   0   0]
 ...
 [  0   0   0 ... 180 182 183]
 [  0   0   2 ... 178 182 181]
 [  0   0   0 ... 178 180 180]]


In [ ]:
score_matrix = matrix(seq_1, seq_2, PAM250, PAM250_index)
print(score_matrix)

In [ ]:
score_matrix

In [ ]:
# #loading in scoring matrix
# file_path = '/Users/SiSi/Documents/GitHub/BMI-203-HW3/BLOSUM50'
# BLOSUM50 = np.fromfile(file_path, dtype=int, sep=" ")#, dtype=float, count=-1, sep='', offset=0)
# with open ('/Users/SiSi/Documents/GitHub/BMI-203-HW3/sequences/prot-0004.fa', "r") as file:
#     seq=file.read()
#     seq = seq[seq.find("\n")+1:].replace('\n', '')


In [19]:
def traceback(H, b, b_='', old_i=0):
    maximum = H.max()
    # flip H to get index of **last** occurrence of H.max() with np.argmax()
    H_flip = np.flip(np.flip(H, 0), 1)
    i_, j_ = np.unravel_index(H_flip.argmax(), H_flip.shape)
    i, j = np.subtract(H.shape, (i_ + 1, j_ + 1))  # (i, j) are **last** indexes of H.max()
    if H[i, j] == 0: #terminating condition
        return b_, j
    b_ = b[j - 1] + '-' + b_ if old_i - i > 1 else b[j - 1] + b_ #True: add gap to sequence; otherwise, add AA
    return traceback(H[0:i, 0:j], b, b_, i)

In [ ]:
traceback(score_matrix, seq_2)

In [20]:
def score(H):
    return H.max()

In [ ]:
score(score_matrix)

In [ ]:
s.strip()

In [ ]:
for f in neg_pairs_filelist:
    a, b = return_sequences(*f)
    

In [25]:
def return_sequences(sequence_a_path, sequence_b_path):
    with open (sequence_a_path, "r") as file:
        a = file.read()
        a = a[a.find("\n")+1:].replace('\n', '')
    with open (sequence_b_path, "r") as file:
        b = file.read()
        b = b[b.find("\n")+1:].replace('\n', '')
    return a, b

In [ ]:
a, b = return_sequences(*neg_pairs_filelist[0])

In [ ]:
print(a)
print(b)

Here I am doing the ROC stuff after implementing S-WM above.

In [29]:
BLOSUM50, BLOSUM50_i = load_sim_matrix('BLOSUM50')

In [30]:
neg_pairs_filelist = []
with open('./Negpairs.txt', mode='r') as my_file:
    for line in my_file:
        neg_pairs_filelist.append(line.strip().split(' '))

In [31]:
neg_pairs_scores = []
for f in neg_pairs_filelist:
    #print(f)
    a, b = return_sequences(*f)
    score_matrix = matrix(a, b, BLOSUM50, BLOSUM50_i, gap_cost=2)
    neg_pairs_scores.append(score(score_matrix))

In [33]:
neg_pairs_scores

[118,
 100,
 64,
 221,
 106,
 128,
 228,
 49,
 117,
 55,
 225,
 71,
 136,
 260,
 162,
 161,
 117,
 80,
 59,
 305,
 320,
 112,
 113,
 276,
 284,
 112,
 198,
 208,
 319,
 135,
 369,
 219,
 168,
 146,
 103,
 120,
 72,
 179,
 127,
 90,
 55,
 77,
 221,
 190,
 313,
 90,
 86,
 41,
 71,
 140]

In [34]:
pos_pairs_filelist = []
with open('./Pospairs.txt', mode='r') as my_file:
    for line in my_file:
        pos_pairs_filelist.append(line.strip().split(' '))

In [56]:
pos_pairs_scores = []
for f in pos_pairs_filelist:
    #print(f)
    a, b = return_sequences(*f)
    score_matrix = matrix(a, b, BLOSUM50, BLOSUM50_i, gap_cost=2)
    pos_pairs_scores.append(score(score_matrix))
pos_pairs_scores = np.array(pos_pairs_scores)

In [57]:
def return_threshold(pos_pairs_scores):
    threshold = np.sort(pos_pairs_scores)[15]
    return threshold

In [58]:
t = return_threshold(pos_pairs_scores)
t

174

In [53]:
sorted_pos_pairs = np.sort(np.array(pos_pairs_scores))
sorted_pos_pairs[15]

174

In [55]:
t = 174
(np.array(pos_pairs_scores) >= t).mean()

0.7

In [52]:
len(pos_pairs_scores)

50

In [43]:
(np.array(neg_pairs_scores) > t).mean()

0.34

In [36]:
pos_pairs_scores

[184,
 172,
 173,
 167,
 246,
 72,
 434,
 118,
 145,
 117,
 157,
 153,
 323,
 209,
 199,
 233,
 180,
 461,
 341,
 452,
 463,
 324,
 271,
 255,
 167,
 234,
 404,
 277,
 223,
 200,
 233,
 246,
 217,
 285,
 359,
 188,
 385,
 238,
 240,
 174,
 154,
 161,
 153,
 301,
 278,
 191,
 458,
 435,
 125,
 108]

https://web.stanford.edu/class/cs262/presentations/lecture2.pdf
https://github.com/applied-bioinformatics/An-Introduction-To-Applied-Bioinformatics/blob/master/book/fundamentals/pairwise-alignment.md